In [ ]:
import numpy as np
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Flatten, Dense, Lambda
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import Xception # TensorFlow ONLY
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
import numpy as np
from tqdm import tqdm 

In [ ]:
## set GPU device
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

In [ ]:
test_image_path = './testing set/'

In [ ]:
image_list = []
import os
def get_data(Dir):
    X = []
    y = []
    for nextDir in os.listdir(Dir):
        if not nextDir.startswith('.'):
            if nextDir in ['FNH']:
                label = 0
            elif nextDir in ['HCA']:
                label = 1
            elif nextDir in ['HGDN']:
                label = 2
            elif nextDir in ['LGDN']:
                label = 3
            elif nextDir in ['WDHCC']:
                label = 4
            elif nextDir in ['NC']:
                label = 5
            elif nextDir in ['NNL']:
                label = 6
                
            temp = Dir + nextDir
                
            for file in tqdm(os.listdir(temp)):
                image = load_img(temp + '/' + file,target_size=(512,512))
                image = img_to_array(image)
                image /= 255
                X.append(image)
                y.append(label)
                image_list.append(file)   
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y

In [ ]:
x_test,y_test = get_data(test_image_path)

In [ ]:
from tensorflow.keras.models import Model, load_model
model_a = load_model('res2_512.h5')

def load_image(img_url):
    image = load_img(img_url,target_size=(512,512))
    image = img_to_array(image)
    image /= 255
    image = np.expand_dims(image,axis=0)
    return image



from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes, normalize=False,title = '', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    
    plt.rcParams['savefig.dpi'] = 300
    plt.rcParams['figure.dpi'] = 300
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
Y_pred = model_a.predict(x_test)
# Convert predictions classes from one hot vectors to labels: [0 0 1 0 0 ...] --> 2
y_pred = np.argmax(Y_pred, axis=1)
# Convert validation observations from one hot vectors to labels
#y_true = np.argmax(Y_valid, axis=1)
y_true = y_test
# compute the confusion matrix
confusion_mtx = confusion_matrix(y_true, y_pred)
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=['FNH','HCA','HGDN','LGDN','WDHCC','NC','NORMAL'])

In [ ]:
con_mat_norm = confusion_mtx.astype('float') / confusion_mtx.sum(axis=1)[:, np.newaxis]     # proportion
con_mat_norm = np.around(con_mat_norm, decimals=2)

plot_confusion_matrix(con_mat_norm, classes=['FNH','HCA','HGDN','LGDN','WDHCC','NC','NORMAL'])

In [ ]:
plt.figure()
plt.plot(fpr, tpr, color='red', linestyle='-.',label='test (AUC = %0.3f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', linestyle=':')
plt.xlim([-0.01, 1.0])
plt.ylim([0.0, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC on External Dataset')
plt.legend(loc="lower right")